In [1]:
import os
import re

from collections import defaultdict

from tqdm.notebook import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from nltk import word_tokenize, sent_tokenize, pos_tag
from nltk.corpus import stopwords
from razdel import sentenize

import pymorphy2 as pm2

In [2]:
def get_or_cache_token_parse(lemmatizer, token, cache):
    """Extracts token parse from cache, or adds it to the latter if not found."""
    if token in cache:
        token_parse = cache[token]
    else:
        token_parse = lemmatizer.parse(token)[0]
        cache[token] = token_parse

    return token_parse

In [3]:
lemmatizer = pm2.MorphAnalyzer(lang='ru')
pm2_cache = {}

Загрузим датасет - корпус научных статей по физике:

In [4]:
data_directory = '../data/elibrary_physics_15k'
article_id = re.compile(r'_([0-9]+).htm$')

file_contents = {}
for filename in tqdm(os.scandir(data_directory), desc='Loading data'):
    if filename.is_file():
        file_name = filename.name
        file_id = int(article_id.search(file_name).groups(1)[0])
        
        with open(filename, "r", encoding='utf-8') as f:
            text = f.read()
            file_contents[file_id] = text

print(f'Размер корпуса: {len(file_contents)} статей')

Loading data: 0it [00:00, ?it/s]

Размер корпуса: 15000 статей


Удалим из корпуса статьи, написанные не на русском языке.

In [5]:
MIN_CYRILLIC_THRESHOLD = 0.2

cyrillic_sym = re.compile(r'[а-яёА-ЯЁ]')

Алгоритм отсеивания статей по языку:
1. подсчитывается кол-во кириллических символов в статье;
2. если соотношение кириллицы >= MIN_CYRILLIC_THRESHOLD -> статья русскоязычная.

In [6]:
rus_articles = {}

for id, article in tqdm(file_contents.items(), desc='Filtering out non-russian articles'):
    rus_count = 0
    for _ in cyrillic_sym.finditer(article):
        rus_count += 1

    if rus_count >= len(article) * MIN_CYRILLIC_THRESHOLD:
        rus_articles[id] = article

print(f'Удалено нерусскоязычных статей: {len(file_contents) - len(rus_articles)}')

Filtering out non-russian articles:   0%|          | 0/15000 [00:00<?, ?it/s]

Удалено нерусскоязычных статей: 4062


In [7]:
articles_count = len(rus_articles)
print(f'Кол-во русскоязычных статей: {articles_count}')

Кол-во русскоязычных статей: 10938


#### 1.2 Arranging lemmatizing procedures

Напишем вспомогательные функции для анализа лемм:

In [8]:
def match_bigram(word1_parse, word2_parse):
    """
    Tries to match bigram of w1 and w2 with any of the patterns.
    If succeed, returns a tuple ((w1_norm, w2_norm), pattern).
    """
    word1_pos = word1_parse.tag.POS
    word2_pos = word2_parse.tag.POS

    # failed pattern matching
    if word2_pos != 'NOUN' or word1_pos not in {'NOUN', 'ADJF'}:
        return None

    word1_inflection = (word1_parse.tag.gender, word1_parse.tag.number, word1_parse.tag.case)
    word2_inflection = (word2_parse.tag.gender, word2_parse.tag.number, word2_parse.tag.case)

    # 'adj + noun' pattern
    if (
        word1_pos == 'ADJF' and word2_pos == 'NOUN' 
        and word1_inflection[1] == word2_inflection[1]
        and word1_inflection[2] == word2_inflection[2]
        and (
            word1_inflection[0] == word2_inflection[0]
            or word1_inflection[0] is None and word1_inflection[1] == 'plur'
        )
    ):
        return (word1_parse.normal_form, word2_parse.normal_form), 'прил. + сущ.'

    # 'noun + noun' pattern
    if word1_pos == 'NOUN' and word2_pos == 'NOUN' and word2_inflection[2] == 'gent':
        return (word1_parse.normal_form, word2_parse.normal_form), 'сущ. + сущ.'

    return None

In [9]:
def simplify_bigram(bigram, lemmatizer, cache):
    """Simplifies bigram and returns its normal form."""
    unigram_1, unigram_2 = tuple(bigram.split(sep='_'))

    normgram_1 = get_or_cache_token_parse(lemmatizer, unigram_1, cache).normal_form
    normgram_2 = get_or_cache_token_parse(lemmatizer, unigram_2, cache).normal_form

    return f'{normgram_1}_{normgram_2}'

Зададим список стоп-слов:

In [10]:
stop_words = stopwords.words('russian')

stop_words.extend(
    [
        # adjective-like pronouns
        'мой', 'твой', 'ваш', 'наш', 'свой', 'его', 'ее', 'их',
        'тот', 'этот', 'такой', 'таков', 'сей', 'который',
        'весь', 'всякий', 'сам', 'самый', 'каждый', 'любой', 'иной', 'другой',
        'какой', 'каков', 'чей', 'никакой', 'ничей',
        'какой-то', 'какой-либо', 'какой-нибудь', 'некоторый', 'некий',
        # participles
        'соответствующий', 'следующий', 'данный',
        # numerals
        'один',
        # insignificant words
        'друг',
    ]
)

stop_words = set(stop_words)

Зададим список стоп-биграмм:

In [11]:
stop_bigrams = []

stop_bigrams.extend(
    [
        ('крайний', 'мера'), ('сегодняшний', 'день'), ('настоящий', 'время'), ('настоящий', 'работа'),
        ('настоящий', 'статья'), ('точка', 'зрение'), ('первый', 'очередь'), ('последний', 'год'),
    ]
)

stop_bigrams = set(stop_bigrams)

Подходящие слова должны содержать только буквы и дефисы (последние не в начале/конце слова):

In [12]:
def match_unigram(word):
    """Checks if a word is long enough and contains only letters and dashes."""
    if word.startswith('-') or word.endswith('-'):
        return False

    word_dashless = word.replace('-', '')
    return len(word_dashless) >= 3 and word_dashless.isalpha()

Загрузим датасет предложений с посчитанными метриками:

In [13]:
bigram_mi3_df = pd.read_csv('../data/data_frames/bigram_tfidf_mi3_scores_full.csv', index_col=0)

In [14]:
def has_predicate(lemmatizer, cache, tokens, predicates):
    """Checks if the sentence contains a predicate."""
    for token in tokens:
        token_pos = get_or_cache_token_parse(lemmatizer, token, cache).tag.POS
        if token_pos in predicates:
            return True

    return False

In [15]:
def has_hyphen(tokens):
    """Checks if the sentence has a hyphen and is valid."""
    valid_tokens_count = 0
    
    for token in tokens:
        if len(token) > 2:
            valid_tokens_count += 1
    if valid_tokens_count < 3:
        return False

    hyphens = ('-', '−', '–', '—')
    for x in hyphens:
        if x in tokens:
            return True

    return False

Наложим ограничения на попадание в выборку:

In [16]:
def get_suitability_status(sentence, formulae_pattern, external_link, internal_link, good_symbols):
    """Checks all conditions to include the sentence in the dataset."""
    if (
        # no equations
        formulae_pattern.search(sentence) is None
        # no external links
        and external_link.search(sentence) is None
        # no internal links
        and internal_link.search(sentence) is None
        # no incomplete fragments
        and len(sentence) >= 10
    ):
        bad_symbols = set(sentence.lower()) - good_symbols

        if not bad_symbols:
            # remove the mess in the beginning
            first_letter = re.search(r'[А-ЯA-Z]', sentence)
            if first_letter is not None:
                return first_letter.start()

    return -1

In [17]:
subtitle_pattern = re.compile('(==.*==(\\n)+)+')
clumped_pattern = re.compile('[а-яА-Я0-9]\.[а-яА-Я0-9]')

formulae_pattern = re.compile(r'\n.\n.\n')
external_link_pattern = re.compile(r'\[([0-9])+\]')
internal_link_pattern = re.compile(r'\(([0-9])+\)')

In [18]:
predicates = {'INFN', 'VERB', 'PRTS', 'ADJS'}

Отберем все символы, не встречающиеся в формулах:

In [19]:
good_symbols = {
    'а', 'б', 'в', 'г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 
    'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я',
    'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r',
    's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
    '(', ')', '%', '\n', ' ', ',', '.', '"', '\'', '!', '/', ':', ';', '?', '[', ']', '«', '»',
    '³', '—', '–', '−', '-', '…', 
}

In [20]:
def extract_sentences(text, text_idx, bigram):
    """Extracts classifiable sentences for target bigram from text."""
    sentences = []
    for idx, substr in enumerate(sentenize(text)):
        sentence = substr.text.strip(r'\n')
        
        # removing subtitles
        subtitle = subtitle_pattern.search(sentence)
        if subtitle is not None:
            sentence = sentence[:subtitle.start(0)] + sentence[subtitle.end(0):]
        
        # capital letter check
        if not sentence[0].isupper():
            continue
        
        # removing sentences with newline characters
        if sentence.find('\n') != -1:
            continue

        # removing clumped sentences
        if clumped_pattern.search(sentence) is not None:
            continue

        # deemphasize sentence
        sentence = sentence.replace('́', '')
        
        tokens = word_tokenize(sentence.lower(), language='russian')

        # saving only sentences with target bigram
        has_target_bigram = False
        normalized_tokens = [get_or_cache_token_parse(lemmatizer, x, pm2_cache).normal_form for x in tokens]
        for i in range(len(normalized_tokens) - 1):
            if normalized_tokens[i] == bigram[0] and normalized_tokens[i + 1] == bigram[1]:
                has_target_bigram = True
                break
        if not has_target_bigram:
            continue
        
        # removing sentences without predicates
        has_predicates = True
        
        standardized_tokens = []
        for token in tokens:
            if match_unigram(token):
                standardized_tokens.append(token)

        if (
            not has_hyphen(tokens)
            and not has_predicate(lemmatizer, pm2_cache, standardized_tokens, predicates)
        ):
            has_predicates = False

        # removing formulas and links
        sentence_start = get_suitability_status(sentence, formulae_pattern, external_link_pattern, internal_link_pattern, good_symbols)
        if sentence_start != -1 and has_predicates:
            sentences.append([sentence[sentence_start:], text_idx])

    return sentences

Сохраним подходящие контексты:

In [21]:
article_1 = bigram_mi3_df[(bigram_mi3_df['unigram_1'] == 'электромагнитный') & (bigram_mi3_df['unigram_2'] == 'волна')]['articles_id_list']
article_1 = eval(list(article_1.values)[0])

article_2 = bigram_mi3_df[(bigram_mi3_df['unigram_1'] == 'акустический') & (bigram_mi3_df['unigram_2'] == 'эмиссия')]['articles_id_list']
article_2 = eval(list(article_2.values)[0])

In [22]:
matched_1 = []

for article_id in article_1:
    text = rus_articles[article_id]
    matched_1.extend(extract_sentences(text, article_id, ('электромагнитный', 'волна')))

In [23]:
matched_2 = []

for article_id in article_2:
    text = rus_articles[article_id]
    matched_2.extend(extract_sentences(text, article_id, ('акустический', 'эмиссия')))

In [24]:
df1 = pd.DataFrame(matched_1, columns=['sentence', 'article_id'])
df1

,sentence,article_id
0,Электромагнитные волны в неоднородной гиротроп...,10007924
1,"ВВЕДЕНИЕ Как известно (см., например, монограф...",10008260
2,"R диэлектрической проницаемости, но не равные ...",10008260
3,"Исходя из этого, можно сделать заключение, что...",10008529
4,Домены тока здесь возникают из-за взаимодейств...,10008529
...,...,...
778,"Магноны - это квазичастицы спиновых волн, как ...",9976930
779,Исходя из уравнения движения электронов в поле...,9989558
780,Электромагнитная волна при этом полностью отда...,9989558
781,Высокие уровни напряжения (сотни киловольт) и ...,9989725


In [25]:
df2 = pd.DataFrame(matched_2, columns=['sentence', 'article_id'])
df2

,sentence,article_id
0,Проведение циклов мартенситных превращений в у...,10007974
1,Характерным процессом при циклировании мартенс...,10007974
2,Остается невыясненным характер акустической эм...,10007974
3,В термомеханическом цикле в одном и том же вре...,10007974
4,Результаты экспериментов и их обсуждение Резул...,10007974
...,...,...
171,"Исследования показали, что наноиндентирование ...",9934683
172,Для измерений акустической эмиссии использовал...,9934683
173,"Эксперименты показали, что увеличение нагрузки...",9934683
174,"Результаты измерения твердости, модуля Юнга, а...",9934683


In [26]:
df1.to_csv('../data/data_frames/data_for_eval/электромагнитный_волна.csv')
df2.to_csv('../data/data_frames/data_for_eval/акустический_эмиссия.csv')